In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from dash import Dash, html, dcc
import plotly.express as px
import datetime
from sklearn import model_selection
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf,pacf
from statsmodels.tsa.seasonal import seasonal_decompose
import itertools
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler

In [ ]:
train = pd.read_csv(r'dataset/train.csv', parse_dates=['Date'],index_col='Date')
store = pd.read_csv(r'dataset/store.csv')
test = pd.read_csv(r'dataset/test.csv')

# **Data Cleaning and EDA**

***Train Dataset***

In [ ]:
train.shape

In [ ]:
train.info()

In [ ]:
train.head()



In [ ]:
train.isnull().sum()

In [ ]:
store.info()

In [ ]:
# Extract Year, Month, Day columns
train['Year'] = train.index.year
train['Month'] = train.index.month
train['Day'] = train.index.day
train['WeekofYear'] = train.index.weekofyear
train.head()

In [ ]:
# Create a new feature 'SalesPerCustomer' to measure average sales per customer
train['SalesPerCustomer'] = train['Sales']/train['Customers']
train.sample(10)

In [ ]:
# Fill all missing values in the dataset with 0
train.fillna(value=0,inplace=True)

In [ ]:
#Check for outlier
pd.set_option('display.float_format', lambda x: '%.2f' % x)
train['Sales'].describe()

In [ ]:
plt.figure(figsize=(8,5))
sns.boxplot(data=train, x='Sales')
plt.title('Boxplot of Sales (Outlier Detection)')
plt.xlabel('Sales')
plt.show()


In [ ]:
# Detect outliers in the 'Sales' column using the Interquartile Range (IQR) method
# Q1: 25th percentile, Q3: 75th percentile
# IQR = Q3 - Q1
# Any value below (Q1 - 1.5*IQR) or above (Q3 + 1.5*IQR) is considered an outlier

Q1 = train['Sales'].quantile(0.25)
Q3 = train['Sales'].quantile(0.75)
IQR = Q3 - Q1

# Calculate lower and upper bounds for detecting outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out the rows that have outlier values in Sales
outliers = train[(train['Sales'] < lower_bound) | (train['Sales'] > upper_bound)]

# Display the number and percentage of outliers
print(f"Number of outliers: {len(outliers)}")
print(f"Percentage of outliers: {len(outliers)/len(train)*100:.2f}%")


***Store Dataset***

In [ ]:
store.shape

In [ ]:
store.info()

In [ ]:
store.head(10)

In [ ]:
store.isnull().sum()

####**Data Merging**

In [ ]:

store_merged = pd.merge(train ,store, on = 'Store', how = 'left')


In [ ]:
store_merged.head(10)

In [ ]:
store_merged.info()

In [ ]:
store_merged['WeekofYear'].nunique()

In [ ]:
store_merged['StoreType'].unique()

In [ ]:
store_merged['StateHoliday'].unique()

In [ ]:
store_merged['StoreType'].unique() 

In [ ]:
store_merged['Assortment'].unique()  

In [ ]:
store_merged['PromoInterval'].unique()  

In [ ]:
#asking data is their any null values
store_merged.isnull().sum()

###**Handling missing data through imputation**

In [ ]:
store_merged['PromoInterval'].fillna('0',inplace=True)

In [ ]:
store_merged['PromoInterval'] = store_merged['PromoInterval'].astype(str)

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 
          'Jul', 'Aug', 'Sept', 'Oct', 'Nov', 'Dec']

for month in months:
    store_merged[f'Promo_{month}'] = store_merged['PromoInterval'].apply(lambda x: 1 if month in x else 0)

In [ ]:
store_merged.drop(columns=['PromoInterval'], inplace=True)

In [ ]:
store_merged.sample(10)

In [ ]:
# Handle CompetitionDistance
store_merged['CompetitionDistance'] = store_merged['CompetitionDistance'].fillna(200000)

# Logical zeros for stores with no competition or promo
store_merged.loc[store_merged['CompetitionDistance'] == 200000, 
           ['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear']] = 0

store_merged.loc[store_merged['Promo2'] == 0, 
           ['Promo2SinceWeek', 'Promo2SinceYear']] = 0

# Iterative imputation for remaining valid gaps
cols_to_impute = ['CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear',
                  'Promo2SinceWeek', 'Promo2SinceYear']

imputer = IterativeImputer(random_state=42)
store_merged[cols_to_impute] = imputer.fit_transform(store_merged[cols_to_impute])


In [ ]:
store_merged['CompetitionOpenSinceYear'].sample(10)

In [ ]:
store_merged['CompetitionOpenSinceMonth'].sample(10)

In [ ]:
store_merged['CompetitionOpenSinceMonth'] = store_merged['CompetitionOpenSinceMonth'].astype(int)
store_merged['CompetitionOpenSinceYear'] = store_merged['CompetitionOpenSinceYear'].astype(int)

In [ ]:
store_merged['CompetitionOpenSinceMonth'].sample(10)

In [ ]:
store_merged['CompetitionOpenSinceMonth'].sample(10)

In [ ]:
# Clean StateHoliday column
# Replace the string '0' with numeric 0 to standardize data types
store_merged['StateHoliday'].replace(to_replace='0',value=0,inplace=True)
store_merged["StateHoliday"].unique()

In [ ]:
# StateHoliday , StoreType , Assortment are still category columns and need to transform with one hot encoding/label encoder for training.
store_merged.info()

In [ ]:
store_merged.info()

####**How many days sales equal zero.**

In [ ]:
#check how many sales value has zero
print(store_merged[store_merged['Sales'] == 0].shape[0])

In [ ]:
store_merged.sample(10)

In [ ]:
#investigate the reason why we have many sales equal zero
zero_sales = store_merged[store_merged['Sales'] == 0]
group_days = zero_sales.groupby('DayOfWeek')['Sales'].size().reset_index(name='ZeroSalesCount')
group_days

In [ ]:
# Group zero sales by DayOfWeek
zero_sales = store_merged[store_merged['Sales'] == 0]
group_days = zero_sales.groupby('DayOfWeek')['Sales'].size().reset_index(name='ZeroSalesCount')

# Sort values by day to make the chart more readable
group_days = group_days.sort_values(by='DayOfWeek')

# Plot bar chart
plt.figure(figsize=(10,6))
sns.barplot(x='DayOfWeek', y='ZeroSalesCount', data=group_days, palette='viridis')

plt.title('Number of Zero Sales per Day of the Week', fontsize=14)
plt.xlabel('Day of Week', fontsize=12)
plt.ylabel('Zero Sales Count', fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()


We found that the maximum day with highest zero sales was the 7th day

# **General plots and Visualizations**
- Sales distribution by year
- Monthly trend sales by year.
- Total sales by month.
- Average sales per day of the week.
- Daily sales trend
- Effect of Promotions on Average Sales.
- Total Sales by Store Type.
- The impact of the state holidays on average sales.
- The number of open and closed days for each store type
- Correlation Heatmap
- Number of stores by year.
- Number of stores opened each month.

In [ ]:
#Sales distribution by year
fig = px.pie(store_merged, values='Sales', names='Year',title="Sales by year")
fig.show()

Stores are open more often in 2013 and 2014 than 2015 which is even more explained in the sales being highest at 2013 and lowest at 2015.

In [ ]:
# Monthly trend sales by year
monthly_sales = store_merged.groupby(['Year','Month'])['Sales'].sum().reset_index()
plt.figure(figsize=(14,5))
sns.lineplot(data=monthly_sales, x='Month', y='Sales', hue='Year', marker='o')
plt.title(' Monthly Sales Trend by Year')
plt.xlabel('Month')
plt.ylabel('Total Sales')
plt.show()


In [ ]:
# Total sales by month to identify seasonal patterns

plt.figure(figsize=(6,5))
store_merged.groupby('Month')['Sales'].sum().plot(kind='bar', color='skyblue')
plt.title('Total Sales by Month', fontsize=14)
plt.ylabel('Total Sales')
plt.xlabel('Month')
plt.xticks(rotation=0)
plt.show()


In [ ]:
fig = px.pie(store_merged, values='Sales', names='Month',title='sales per month')
fig.show()

Sales are highest in march and lowest in septemper and just like said previously sales decrease in the last 5 months beginning with august.

In [ ]:
# Average sales per day of the week
##“Which days of the week have higher average sales?”

plt.figure(figsize=(6,5))
store_merged.groupby('DayOfWeek')['Sales'].mean().plot(kind='bar', color='lightgreen')
plt.title('Average Sales by Day of the Week', fontsize=14)
plt.ylabel('Average Sales')
plt.xlabel('Day of the Week')
plt.xticks(rotation=0)
plt.show()


In [ ]:
# Daily sales trend
daily_sales = store_merged.groupby('Day')['Sales'].sum().reset_index()
plt.figure(figsize=(14,5))
sns.lineplot(data=daily_sales, x='Day', y='Sales')
plt.title(' Daily Sales Trend')
plt.xlabel('Day')
plt.ylabel('Total Sales')
plt.show()



## Compare sales with and without promo

In [ ]:
promo_sales = store_merged.groupby('Promo')['Sales'].mean().reset_index()
print(promo_sales)

In [ ]:

plt.figure(figsize=(6,5))
sns.barplot(data=promo_sales, x='Promo', y='Sales', palette='viridis')
plt.title('Effect of Promotions on Average Sales')
plt.ylabel('Average Sales')
plt.xticks([0, 1], ['No Promo', 'Promo'])
plt.show()

### Total Sales by Store Type
 **“Which store type contributes most to sales?”**  


- `a` → Large stores (hypermarkets or big supermarkets) with high sales.  
- `b` → Medium-sized supermarkets with moderate traffic.  
- `c` → Small neighborhood stores with lower sales but frequent customers.  
- `d` → Specialized or discount stores with varying performance.


In [ ]:
# Group the data by StoreType and sum Sales, Customers, and SalesPerCustomer
plt.figure(figsize=(6,5))
store_merged.groupby('StoreType')[['Sales', 'Customers', 'SalesPerCustomer']]\
    .sum().sort_values('Sales', ascending=False)\
    .plot(kind='bar', figsize=(10,6))
plt.title('Sales by Store Type', fontsize=14)
plt.ylabel('Total')
plt.xlabel('Store Type')
plt.xticks(rotation=0)
plt.show()



we see that store type a has the most number of sales and customers while store type b has the least

In [ ]:
#  bar plot for total sales by store type for a clearer view.
store_type_sales = store_merged.groupby('StoreType')['Sales'].sum().reset_index().sort_values('Sales', ascending=False)

plt.figure(figsize=(6,5))
sns.barplot(data=store_type_sales, x='StoreType', y='Sales', palette='Blues')
plt.title('Total Sales by Store Type')
plt.xlabel('Store Type')
plt.ylabel('Total Sales')
plt.show()

In [ ]:
#Which stores have the highest and lowest sales?
store_merged.groupby('StoreType')[['Customers', 'Sales', 'SalesPerCustomer']].sum().sort_values('Sales', ascending=False)



###  The impact of state holidays on average sales to understand seasonal

- `0` → No holiday (a regular business day).
- `a` → Public holiday (e.g., national or state-level holidays).
- `b`→ Easter holiday (a special holiday period around Easter).
- `c`→ Christmas holiday (the holiday period around Christmas).

In [ ]:
# Visualization of the impact of state holidays on average sales .

plt.figure(figsize=(8,5))
sns.barplot(data=store_merged, x='StateHoliday', y='Sales', estimator='mean', palette='viridis')
plt.title('Average Sales by State Holiday Type')
plt.xlabel('State Holiday Type')
plt.ylabel('Average Sales')
plt.show()

In [ ]:
#Showing number of the days where the store is opened and the sales = 0.
store_merged[(store_merged.Open == 1) & (store_merged.Sales == 0)].shape[0]

There is a small number of days in which the store was open and yet there were no sales.

**Count the number of open and closed days for each store type**

In [ ]:
open_days = store_merged.groupby(['StoreType', 'Open']).size().reset_index(name='Days')
open_days.head(10)

In [ ]:
plt.figure(figsize=(8,5))
sns.barplot(data=open_days, x='StoreType', y='Days', hue='Open')

plt.title('Days Open vs Closed by Store Type')
plt.xlabel('Store Type')
plt.ylabel('Number of Days')
plt.legend(title='Open', labels=['Closed (0)', 'Open (1)'])
plt.show()

we can see that store b had the least number of open days and is far less represented in the dataset compared to the other stores followed closely by store c

In [ ]:
#  Correlation Heatmap

##“What variables are most correlated with sales?”

plt.figure(figsize=(10,6))
corr_matrix = store_merged[['Sales','Customers','SalesPerCustomer','Promo']].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap', fontsize=14)
plt.show()


In [ ]:
#Number of stores opened by year
fig = px.pie(store_merged, values='Open', names='Year')
fig.show()

stores have been opening less each year with the highest number of store opened in 2013 and lowest being in 2015

In [ ]:
#Number of stores opened each month.
plt.figure(figsize=(10, 6));
sns.countplot(x='Month', hue='Open', data=store_merged);
plt.title("stores open each month")
plt.show()

stores open on average consistently each month with the exception of the last 5 months where there is a notable drop off in general datapoints.

###  preprocessing data for training 


In [ ]:
# Define numeric and categorical columns
numeric_cols = [
    'Sales', 'Customers', 'Day', 'Month',
    'CompetitionDistance', 'CompetitionOpenSinceMonth', 'CompetitionOpenSinceYear',
    'Promo2SinceWeek', 'Promo2SinceYear', 'Year','WeekofYear','DayOfWeek'
]

categorical_cols = ['StoreType', 'Assortment', 'StateHoliday']

In [ ]:
store_merged = pd.get_dummies(store_merged, columns=categorical_cols, prefix=categorical_cols) 

In [ ]:
store_merged_scaled = store_merged.copy()
scaler = MinMaxScaler()
store_merged_scaled[numeric_cols] = scaler.fit_transform(store_merged_scaled[numeric_cols])

In [ ]:
store_merged_scaled.sample(10)

In [ ]:
store_merged_scaled.info()

In [ ]:
store_merged[numeric_cols] = store_merged[numeric_cols].astype('float32')
store_merged = store_merged.astype({col: 'float32' for col in store_merged.select_dtypes('uint8').columns})
store_merged_scaled[numeric_cols] = store_merged_scaled[numeric_cols].astype('float32')
store_merged_scaled = store_merged_scaled.astype({col: 'float32' for col in store_merged_scaled.select_dtypes('uint8').columns})

In [ ]:
store_merged_scaled.columns

In [ ]:
drop_cols = ['Store', 'Sales', 'Customers', 'SalesPerCustomer']
X = store_merged.drop(columns=drop_cols)
y = store_merged['Sales']


In [ ]:
print(X.shape,y.shape)

###  Testing data for stationarity 


In [ ]:
sales_a = train[train.Store == 2]['Sales']
sales_b = train[train.Store == 85]['Sales'].sort_index(ascending = True) 
sales_c = train[train.Store == 1]['Sales']
sales_d = train[train.Store == 13]['Sales']

frame, (ax1, ax2, ax3, ax4) = plt.subplots(4, figsize = (20, 16))

# Visualize Trend 
sales_a.resample('w').sum().plot(ax = ax1)
sales_b.resample('w').sum().plot(ax = ax2)
sales_c.resample('w').sum().plot(ax = ax3)
sales_d.resample('w').sum().plot(ax = ax4)


In [ ]:
def test_stationarity(timeseries):
    # Determine rolling statestics 
    roll_mean = timeseries.rolling(window=7).mean()
    roll_std = timeseries.rolling(window=7).std()
    
    # plotting rolling statestics 
    plt.subplots(figsize = (16, 6))
    orginal = plt.plot(timeseries.resample('w').mean(), color='blue',linewidth= 3, label='Orginal')
    roll_mean = plt.plot(roll_mean.resample('w').mean(), color='red',linewidth= 3, label='Rolling Mean')
    roll_mean = plt.plot(roll_std.resample('w').mean(), color='green',linewidth= 3, label='Rolling Std')
    
    plt.legend(loc='best')
    plt.show()
    
    # Performing Dickey-Fuller test 
    print('Results of Dickey-Fuller test:')
    result= adfuller(timeseries, autolag='AIC')
    
    print('ADF Statistics: %f' %result[0])
    print('P-value: %f' %result[1])
    print('Critical Values:')
    for key, value in result[4].items():
        print(key, value)

In [ ]:
test_stationarity(sales_a)

In [ ]:
test_stationarity(sales_b)

In [ ]:
test_stationarity(sales_c)

In [ ]:
test_stationarity(sales_d)

From above charts we could deduct that, mean and variance of the data do not change most of the time. So, we do not compute any transformation as the data is already stationary.


In [ ]:
def plot_timeseries(sales,StoreType):

    fig, axes = plt.subplots(2, 1, sharex=True, sharey=False)
    fig.set_figheight(6)
    fig.set_figwidth(20)

    decomposition= seasonal_decompose(sales, model = 'additive',period=365)

    estimated_trend = decomposition.trend
    estimated_seasonal = decomposition.seasonal
    estimated_residual = decomposition.resid
    
    axes[1].plot(estimated_seasonal, 'g', label='Seasonality')
    axes[1].legend(loc='upper left');
    
    axes[0].plot(estimated_trend, label='Trend')
    axes[0].legend(loc='upper left');

    plt.title('Decomposition Plots')

In [ ]:
plot_timeseries(sales_a, 'a')

In [ ]:
plot_timeseries(sales_b, 'b')

In [ ]:
plot_timeseries(sales_c, 'c')

In [ ]:
plot_timeseries(sales_d, 'd')

From the above plots, we can see that there is seasonality and trends present in our data. So, we'll use forecasting models that take both of these factors into consideration.

In [ ]:
def auto_corr(sales):
    lag_acf = acf(sales, nlags=30)
    lag_pacf = pacf(sales, nlags=20, method='ols')
    
    plt.figure(figsize=(10,5))
    
    plt.subplot(121)
    plt.plot(lag_acf)
    plt.axhline(y=0, linestyle='--', color='red')
    plt.axhline(y=1.96/np.sqrt(len(sales)), linestyle='--', color='red')
    plt.axhline(y=-1.96/np.sqrt(len(sales)), linestyle='--', color='red')
    plt.title('Autocorrelation (ACF)')
    
    plt.subplot(122)
    plt.plot(lag_pacf)
    plt.axhline(y=0, linestyle='--', color='red')
    plt.axhline(y=1.96/np.sqrt(len(sales)), linestyle='--', color='red')
    plt.axhline(y=-1.96/np.sqrt(len(sales)), linestyle='--', color='red')
    plt.title('Partial Autocorrelation (PACF)')
    
    plt.tight_layout()
    plt.show()


In [ ]:
auto_corr(sales_a)


In [ ]:
auto_corr(sales_b)

In [ ]:
auto_corr(sales_c)

In [ ]:
auto_corr(sales_d)

In [ ]:
train_arima = train.resample('w').mean()
train_arima = train_arima[['Sales']]
train_arima.plot()

### First model : SARIMAX

In [ ]:
# Define the p, d and q parameters to take any value between 0 and 3
p = d = q = range(0, 2)

# Generate all different combinations of p, q and q triplets
pdq = list(itertools.product(p, d, q))

# Generate all different combinations of seasonal p, q and q triplets
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

print('Examples of parameter combinations for Seasonal ARIMA: ')
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[1]))
print('SARIMAX: {} x {}'.format(pdq[1], seasonal_pdq[2]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[3]))
print('SARIMAX: {} x {}'.format(pdq[2], seasonal_pdq[4]))

In [ ]:
# Determing p,d,q combinations with AIC scores.
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(train_arima,
                                            order=param,
                                            seasonal_order=param_seasonal,
                                            enforce_stationarity=False,
                                            enforce_invertibility=False)

            results = mod.fit()

            print('ARIMA{}x{}12 - AIC:{}'.format(param, param_seasonal, results.aic))
        except:
            continue

In [ ]:
# Fitting the data to SARIMA model 
model_sarima = sm.tsa.statespace.SARIMAX(train_arima,
                                        order=(1, 1, 1),
                                        seasonal_order=(1,1,1,12),
                                        enforce_stationarity=False,
                                        enforce_invertibility=False)
results_sarima= model_sarima.fit()
print(results_sarima.summary().tables[1])

In [ ]:
# Checking diagnostic plots
results_sarima.plot_diagnostics(figsize=(16, 10))
plt.show()

In [ ]:
from math import sqrt
# Model prediction 

pred = results_sarima.get_prediction(start=pd.to_datetime('2015-1-4'), dynamic=False)

# Get confidence interval of forecast 
pred_ci = pred.conf_int()

ax = train_arima['2014':].plot(label='Observed', figsize=(15,7))
pred.predicted_mean.plot(ax=ax, label='One step ahed Forecast', alpha=1)

ax.fill_between(pred_ci.index, 
               pred_ci.iloc[:, 0],
               pred_ci.iloc[:,1],
               color='r', alpha=.1)

ax.set_xlabel('Date')
ax.set_ylabel('Sales')
plt.legend()
plt.show()

train_arima_forecasted = pred.predicted_mean
train_arima_truth = train_arima['2015-01-04':]

rms_arima= sqrt(mean_squared_error(train_arima_truth,train_arima_forecasted))
print('Root Mean Squared Error = ',rms_arima)

### Second model : XGBoost

In [ ]:
ts_xgboost = store_merged.copy()
ts_xgboost = ts_xgboost.drop(['Customers', 'SalesPerCustomer'], axis=1)

In [ ]:
ts_xgboost['CompetitionDistance'] = np.log1p(ts_xgboost['CompetitionDistance'])

In [ ]:
ts_xgboost['CompetitionOpen'] = 12 * (ts_xgboost.Year - ts_xgboost.CompetitionOpenSinceYear) + (ts_xgboost.Month - ts_xgboost.CompetitionOpenSinceMonth)
ts_xgboost['PromoOpen'] = 12 * (ts_xgboost.Year - ts_xgboost.Promo2SinceYear) + (ts_xgboost.WeekofYear - ts_xgboost.Promo2SinceWeek) / 4.0
ts_xgboost = ts_xgboost.drop(["CompetitionOpenSinceMonth", "CompetitionOpenSinceYear"], axis = 1)
ts_xgboost = ts_xgboost.drop(["Promo2SinceWeek", "Promo2SinceYear"], axis = 1)

In [ ]:
ts_xgboost.info()

In [ ]:
features = ts_xgboost.drop(["Sales"], axis = 1)
target = ts_xgboost["Sales"]

X_train, X_test, y_train, y_test = model_selection.train_test_split(features, target, test_size = 0.20)

In [ ]:
from xgboost import XGBRegressor
import xgboost as xgb
# Tuning parameters - using default metrics
params = {'max_depth':6, "booster": "gbtree", 'eta':0.3, 'objective':'reg:linear'} 

dtrain = xgb.DMatrix(X_train, y_train)
dtest = xgb.DMatrix(X_test, y_test)
watchlist = [(dtrain, 'train'), (dtest, 'eval')]

# Training the model
xgboost = xgb.train(params, dtrain, 100, evals=watchlist,early_stopping_rounds= 100, verbose_eval=True)
         
# Making predictions
preds = xgboost.predict(dtest)

In [ ]:
rms_xgboost = sqrt(mean_squared_error(y_test, preds))
print("Root Mean Squared Error for XGBoost:", rms_xgboost)

### SARIMA RMSE = 867.955 , XGBoost RMSE = 1170.682